In [1]:
import os
import pandas as pd
import numpy as np
from os import path
import glob

In [4]:
red=False
blue=False
if red == True:
    print('red is true')
elif blue == True:
    print('blue is true')

In [2]:
import os
red=True
blue=False
if (red==True) and (blue==True):
    raise ValueError("cant be both red and blue")    

In [2]:
chs = pd.read_csv('/scratch/Shares/dowell/genomes/hg38/hg38.chrom.sizes', sep='\t', header=None)
chs.columns = ['chr', 'counts']
chs['counts'].mean()
chs.describe()

,counts
count,1.950000e+02
mean,1.589704e+07
std,4.680446e+07
min,9.700000e+02
25%,1.997000e+03
50%,3.203200e+04
75%,1.667610e+05
max,2.489564e+08


In [26]:
def fimotobed(outdir):
    def identifier(row):
        return str(row['sequence_name']) + ':' + str(row['start']) + '-' + str(row['stop']) + '_' + str(row['strand'] + ';' + str(row['motif_id']))

    def percent_gc(row):
        r = pd.Series(row['matched_sequence'])
        z = r.str.count('G') + r.str.count('C')
        x = z/len(row['matched_sequence'])
        return x

    motif_dirs = glob.glob(outdir + '/temp/sim_fimo_out/*')
    for motif_dir in motif_dirs:
        if path.isdir(motif_dir):
            motif_name = motif_dir.split('/')[-1]
            print(motif_name)
            df = pd.read_csv('%s/fimo.tsv' % (motif_dir), sep ='\t')
            print("Post-processing FIMO output for %s" % motif_name)
            df.drop(df.tail(3).index,inplace=True)
            if df.empty == True:
                print('Skipping ' + motif_name + ' -no motif hits.')
            else:
                df = df.sort_values('sequence_name').reset_index()
                df['identifier'] = df.apply(lambda row: identifier(row), axis=1)
                df['percent_gc'] = df.apply(lambda row: percent_gc(row), axis=1)
                df['mean_percent_gc'] = df['percent_gc'].mean()
                df = df[['sequence_name', 'start', 'stop', 'identifier', 'score', 'strand', 
                         'matched_sequence', 'percent_gc', 'mean_percent_gc']]
                df.to_csv(outdir + '/motifs/simulated/' + motif_name + '.sorted.bed', sep='\t', header=None, index=False)

def gc_content(outdir):
    directory = os.fsencode(outdir + '/motifs/simulated')
    output = pd.DataFrame({'identifier' : [], 'mean_percent_gc' : []})
    for file in os.listdir(directory):
        if file.endswith(b'.bed'):
            file = file.decode('utf-8')
            df = pd.read_csv(outdir + '/motifs/simulated/' + file, sep="\t", header = None, 
                             names = ['sequence_name', 'start', 'stop', 'identifier', 'score', 'strand', 'matched_sequence', 'percent_gc', 'mean_percent_gc'])
            df = df[['identifier', 'mean_percent_gc']].loc[0]
            output=output.append(df)
            output.to_csv(outdir + '/annotations/simulated_mean_gc_percent.txt', sep="\t", header=None, index=False)
        else:
            continue


In [28]:
#fimotobed(outdir = '/scratch/Users/tajo5912/rbg/test')
gc_content(outdir = '/scratch/Users/tajo5912/rbg/test')

In [30]:
### Testing functions for fimo formating
# fimoout = '/scratch/Users/tajo5912/rbg/test/temp/fimo'
# bedout = '/scratch/Users/tajo5912/rbg/test/temp/bed'

# def identifier(row):
#     return str(row['sequence_name']) + ':' + str(row['start']) + '-' + str(row['stop']) + '_' + str(row['strand'] + ';' + str(row['motif_id']))

# def percent_gc(row):
#     r = pd.Series(row['matched_sequence'])
#     z = r.str.count('G') + r.str.count('C')
#     x = z/len(row['matched_sequence'])
#     return x

# motif_dirs = glob.glob(fimoout + '/*')
# for motif_dir in motif_dirs:
#     if path.isdir(motif_dir):
#         motif_name = motif_dir.split('/')[-1]
#         print(motif_name)
#         df = pd.read_csv('%s/fimo.tsv' % (motif_dir), sep ='\t')
#         print("Post-processing FIMO output for %s" % motif_name)
#         df.drop(df.tail(3).index,inplace=True)
#         if df.empty == True:
#             print('Skipping ' + motif_name + ' -no motif hits.')
#         else:
#             df = df.sort_values('sequence_name').reset_index()
#             df['identifier'] = df.apply(lambda row: identifier(row), axis=1)
#             df['percent_gc'] = df.apply(lambda row: percent_gc(row), axis=1)
#             df['mean_percent_gc'] = df['percent_gc'].mean()
#             df = df[['sequence_name', 'start', 'stop', 'identifier', 'score', 'strand', 
#                      'matched_sequence', 'percent_gc', 'mean_percent_gc']]
#             df.to_csv(bedout + '/' + motif_name + '.sorted.bed', sep='\t', header=None, index=False)

In [19]:
gata = '/scratch/Users/tajo5912/rbg/test/temp/fimo/GATA3_M05752_1/fimo.tsv'
gata = pd.read_csv(gata, sep='\t')
gata.drop(gata.tail(3).index,inplace=True)
if gata.empty == True:
    print('skip')
else:
    df['identifier'] = df.apply(lambda row: identifier(row), axis=1)
    df['percent_gc'] = df.apply(lambda row: percent_gc(row), axis=1)

skip


In [37]:
outdir='/scratch/Users/tajo5912/rbg/test'
directory = os.fsencode(outdir + '/motifs/simulated')
output = pd.DataFrame({'identifier' : [], 'mean_percent_gc' : []})
for file in os.listdir(directory):
    if file.endswith(b'.bed'):
        file = file.decode('utf-8')
        df = pd.read_csv(outdir + '/motifs/simulated/' + file, sep="\t", header = None, 
                         names = ['sequence_name', 'start', 'stop', 'identifier', 'score', 'strand', 'matched_sequence', 'percent_gc', 'mean_percent_gc'])
        df = df[['identifier', 'mean_percent_gc']].loc[0]
        output=output.append(df)
        output.to_csv(outdir + '/annotations/simulated_mean_gc_percent.txt', sep="\t", header=None, index=False)
    else:
        continue


In [8]:
def chrm_sizes_whole_genome(genome, outdir):
    g = genome.split('/')[-1].split('.')[-2]
    print(g)
    try:
        os.system("cut -f1,2 " + genome + ".fai > " + outdir + "/generated_sequences/" + g + ".chrom.sizes")
    except OSError:
        print ("Creation of chromosome size file %s failed" % outdir + "/generated_sequences/" + g + ".chrom.sizes")
    else:
        print ("Successfully created chromosome size file %s" % outdir + "/generated_sequences/" + g + ".chrom.sizes")
            
chrm_sizes_whole_genome(genome='/scratch/Shares/dowell/genomes/hg38/hg38.fa', 
                        outdir='/scratch/Users/tajo5912/rbg/test')

hg38
Successfully created chromosome size file /scratch/Users/tajo5912/rbg/test/generated_sequences/hg38.chrom.sizes


In [9]:
os.stat('/scratch/Users/tajo5912/rbg/test/annotations/test_simulated_window.bed')

os.stat_result(st_mode=33188, st_ino=9088365749575586314, st_dev=41, st_nlink=1, st_uid=573627, st_gid=1100000, st_size=103100, st_atime=1612546485, st_mtime=1612546485, st_ctime=1612546485)

In [ ]:
#Adapted from DAStk by Ignacio


import csv
import datetime
import glob
import multiprocessing
import numpy as np
import os.path
import pandas as pd
import sys
import matplotlib as mpl
import pybedtools
# to prevent DISPLAY weirdness when running in the cluster:
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.ioff()
from functools import partial
from operator import itemgetter
#from functools import reduce


# around the given mean
def is_in_window(motif_interval, bed_median, window_size):
    start = int(motif_interval.start)
    end = int(motif_interval.end)
    if (end >= (bed_median - window_size) and end <= (bed_median + window_size)) \
        or (start >= (bed_median - window_size) and start <= (bed_median + window_size)):
        return True
    else:
        return False
    


def find_motifs_in_chrom(current_chrom, files):
    tf_motif_filename, windowbed_name, window = files
    H = window          # in bps, the MD-score parameter (large window)
    h = window * 0.1           # in bps, the MD-score parameter (small window)

    wdf = pd.read_csv(windowbed_name, header=None, comment='#', sep="\t", usecols=[0, 1, 2], \
                          names=['chrom', 'start', 'end'], na_filter=False, dtype={'chrom':'str', 'start':'int', 'end':'int'})
    w_iter = wdf[(wdf.chrom == current_chrom)].itertuples()
    motif_df = pd.read_csv(tf_motif_filename, header=None, comment='#', sep="\t", usecols=[0, 1, 2], \
                           names=['chrom', 'start', 'end'], na_filter=False, dtype={'chrom':'str', 'start':'int', 'end':'int'})
    if len(motif_df) == 0:
        return None
    motif_iter = motif_df[(motif_df.chrom == current_chrom)].itertuples()
    last_motif = None
    g_h = 0
    g_H = 0
    total_motif_sites = 0
    tf_distances = []
    try:
        motif_region = next(motif_iter)   # get the first motif in the list
    except StopIteration:
        print('No motifs for chromosome ' + current_chrom + ' on file ' + tf_motif_filename)
        return None

    peak_count_overlapping_motif = 0
    for windowbed_peak in atac_iter:
        motifs_within_region = True

        bed_median = bed_peak.start + (bed_peak.end - bed_peak.start)/2
        # check the last motif, too, in case any of them falls within the region of
        # interest of two sequential ATAC-Seq peaks
        if last_motif:
            if is_in_window(last_motif, bed_median, h):
                g_h = g_h + 1
            if is_in_window(last_motif, bed_median, H):
                g_H = g_H + 1
                tf_median = last_motif.start + (last_motif.end - last_motif.start)/2
                tf_distances.append(bed_median - tf_median)
                try:
                    motif_region = next(motif_iter)   # get the next motif in the list
                    total_motif_sites += 1
                except StopIteration:
                    pass
                last_motif = motif_region
                if motif_region.start > (bed_median + H):
                    motifs_within_region = False

        # Move to the next putative motif sites until we get one past our evaluation window
        while motifs_within_region:
            total_motif_sites += 1
            # account for those within the smaller window (h)
            if is_in_window(motif_region, bed_median, h):
                g_h = g_h + 1
            # account for those within the larger window (H)
            if is_in_window(motif_region, bed_median, H):
                g_H = g_H + 1
                tf_median = motif_region.start + (motif_region.end - motif_region.start)/2
                tf_distances.append(bed_median - tf_median)

            # if we still haven't shifted past this peak...
            if motif_region.start <= (bed_median + H):
                try:
                    motif_region = next(motif_iter)   # get the next motif in the list
                    total_motif_sites += 1
                except StopIteration:
                    # No more TF motifs for this chromosome
                    break
                last_motif = motif_region
            else:
                motifs_within_region = False

    # Count any remaining TF motif sites after the last peak
    while(len(motif_region) > 0):
        try:
            motif_region = next(motif_iter)   # this gets the next motif in the list
        except StopIteration:
            break

    return [tf_distances, g_h, g_H, total_motif_sites]


#md_score, small_window, large_window, motif_site_count, heat] = 
#et_md_score(filename, int(threads), windowbed_name, CHROMOSOMES, window)





In [ ]:
# MD functions .py
#filename, int(threads), windowbed_name, CHROMOSOMES, window
# around the given mean
def is_in_window(motif_interval, bed_median, window_size):
    start = int(motif_interval.start)
    end = int(motif_interval.end)
    if (end >= (bed_median - window_size) and end <= (bed_median + window_size)) \
        or (start >= (bed_median - window_size) and start <= (bed_median + window_size)):
        return True
    else:
        return False
    


def find_motifs_in_chrom(current_chrom, files):
    tf_motif_filename, windowbed_name, window = files
    H = window          # in bps, the MD-score parameter (large window)
    h = window * 0.1           # in bps, the MD-score parameter (small window)

    wdf = pd.read_csv(windowbed_name, header=None, comment='#', sep="\t", usecols=[0, 1, 2], \
                          names=['chrom', 'start', 'end'], na_filter=False, dtype={'chrom':'str', 'start':'int', 'end':'int'})
    w_iter = wdf[(wdf.chrom == current_chrom)].itertuples()
    motif_df = pd.read_csv(tf_motif_filename, header=None, comment='#', sep="\t", usecols=[0, 1, 2], \
                           names=['chrom', 'start', 'end'], na_filter=False, dtype={'chrom':'str', 'start':'int', 'end':'int'})
    if len(motif_df) == 0:
        return None
    motif_iter = motif_df[(motif_df.chrom == current_chrom)].itertuples()
    last_motif = None
    g_h = 0
    g_H = 0
    total_motif_sites = 0
    tf_distances = []
    try:
        motif_region = next(motif_iter)   # get the first motif in the list
    except StopIteration:
        print('No motifs for chromosome ' + current_chrom + ' on file ' + tf_motif_filename)
        return None

    peak_count_overlapping_motif = 0
    for windowbed_peak in atac_iter:
        motifs_within_region = True

        bed_median = bed_peak.start + (bed_peak.end - bed_peak.start)/2
        # check the last motif, too, in case any of them falls within the region of
        # interest of two sequential ATAC-Seq peaks
        if last_motif:
            if is_in_window(last_motif, bed_median, h):
                g_h = g_h + 1
            if is_in_window(last_motif, bed_median, H):
                g_H = g_H + 1
                tf_median = last_motif.start + (last_motif.end - last_motif.start)/2
                tf_distances.append(bed_median - tf_median)
                try:
                    motif_region = next(motif_iter)   # get the next motif in the list
                    total_motif_sites += 1
                except StopIteration:
                    pass
                last_motif = motif_region
                if motif_region.start > (bed_median + H):
                    motifs_within_region = False

        # Move to the next putative motif sites until we get one past our evaluation window
        while motifs_within_region:
            total_motif_sites += 1
            # account for those within the smaller window (h)
            if is_in_window(motif_region, bed_median, h):
                g_h = g_h + 1
            # account for those within the larger window (H)
            if is_in_window(motif_region, bed_median, H):
                g_H = g_H + 1
                tf_median = motif_region.start + (motif_region.end - motif_region.start)/2
                tf_distances.append(bed_median - tf_median)

            # if we still haven't shifted past this peak...
            if motif_region.start <= (bed_median + H):
                try:
                    motif_region = next(motif_iter)   # get the next motif in the list
                    total_motif_sites += 1
                except StopIteration:
                    # No more TF motifs for this chromosome
                    break
                last_motif = motif_region
            else:
                motifs_within_region = False

    # Count any remaining TF motif sites after the last peak
    while(len(motif_region) > 0):
        try:
            motif_region = next(motif_iter)   # this gets the next motif in the list
        except StopIteration:
            break

    return [tf_distances, g_h, g_H, total_motif_sites]

def get_md_score(tf_motif_filename, threads, windowbed_name, CHROMOSOMES, window):

    HISTOGRAM_BINS = 150
    pool = multiprocessing.Pool(threads)
    results = pool.map(partial(find_motifs_in_chrom, files=[tf_motif_filename, windowbed_name, window]), CHROMOSOMES)
    pool.close()
    pool.join()

    results_matching_motif = [x for x in results if x is not None]
    if len(results_matching_motif) > 0:
        sums = np.sum(results_matching_motif, axis=0)
        overall_g_h = sums[1]
        overall_g_H = sums[2]
        overall_motif_sites = sums[3]

        # Calculate the heatmap for this motif's barcode
        tf_distances = sums[0]
        heatmap, xedges = np.histogram(tf_distances, bins=HISTOGRAM_BINS)
        str_heatmap = np.char.mod('%d', heatmap)
        if overall_g_H >= 0:
            return [float(overall_g_h + 1)/(overall_g_H + 1), (overall_g_h + 1), (overall_g_H + 1), (overall_motif_sites + 1), ';'.join(str_heatmap)]
    else:
        return None

In [ ]:
# ### Run this as main, drop MD functions and only use MDScore class for pooling?
# import os
# import sys
# from os import path
# import pandas as pd
# import csv

# class MDScore:
#     '''Setting up for MD-Score calculations'''
#     def __init__(self, window, threads, outdir, sample):
#         self.window = window
#         self.threads = threads
#         self.outdir = outdir
#         self.sample = sample   

#     def mdcalc(self):
#         windowbed_name = self.outdir + '/annotations/' + self.sample + '_simulated_window.bed' 
#         input_file_statinfo = os.stat(windowbed_name)    
#         if input_file_statinfo.st_size !=0:
#             windowbed = open(windowbed_name)
#             windowbed_csv_reader = csv.reader(windowbed, delimiter='\t')
#             windowbed_line = next(windowbed_csv_reader)
#             windowbed_peak_count = 0        
#         else:
#             raise ValueError("The windowed annotation file (sample_simulated_window.bed) is empty!")
        
#         chromosomes = self.outdir + '/generated_sequences/' + self.sample + '_simulated.chrom.sizes'
#         if chromosomes:
#             chr_df = pd.read_csv(chromosomes, header=None, comment='#', sep="\t", usecols=[0,1], \
#                                       names=['chrom', 'size'], na_filter=False, dtype={'chrom':'str', 'size':'int'})
#             chr_list = list(chr_df['chrom'])
#             CHROMOSOMES = [word for word in chr_list if len(word) <= 6]
#         else:
#             raise ValueError("Chromosome size file missing from generated_sequences(sample_simulated.chrom.sizes)!") 
            
#         windowbed.seek(0)
#         windowbed_csv_reader = csv.reader(windowbed, delimiter='\t')
#         windowbed_line = next(windowbed_csv_reader)
#         while(windowbed_line[0][0] == '#'):
#             windowbed_line = next(windowbed_csv_reader)    
        
#         motif_stats = []
#         sorted_motif_stats = []
#         tf_motif_path = self.outdir + '/motifs/simulated/*'
#         motif_filenames = glob.glob(tf_motif_path)
#         motif_count = len(motif_filenames)
#         print("Processing %s motif files in %s" % motif_count, tf_motif_path)
#         for filename in motif_filenames:
#             filename_no_path = filename.split('/')[-1]
#             if os.path.getsize(filename) > 0 and os.path.basename(filename).endswith(tuple(['.bed'])):
#                 [md_score, small_window, large_window, motif_site_count, heat] = get_md_score(filename, int(self.threads), windowbed_name, CHROMOSOMES, self.window)
#                 print('The MD-score for the provided BED regions vs %s is %.6f' % (filename_no_path, md_score))
#                 motif_stats.append({ 'motif_file': filename_no_path,'md_score': md_score, 'small_window': small_window, 'large_window': large_window, 'motif_site_count': motif_site_count, 'heat': heat })
#         # sort the stats dictionary by MD-score, descending order
#         sorted_motif_stats = sorted(motif_stats, key=itemgetter('md_score'), reverse=True)

#         md_score_fp = open("%s/final_output/%s_simulated_md_scores.txt" % (self.outdir, self.sample), 'w')
#         for stat in sorted_motif_stats:
#             md_score_fp.write("%s,%s,%s,%s,%s,%s\n" % (stat['motif_file'], stat['md_score'], stat['small_window'], \
#                                stat['large_window'], stat['motif_site_count'], stat['heat']))
#         md_score_fp.close()

In [87]:
sequence_num = 5073
window = 1500
chrom_num=10

#sequence_num = len(file)
seq_length = ((2*int(window))+21)
tot_bases = int(sequence_num*seq_length)

seq_per_chrom = math.floor(sequence_num/chrom_num)
seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
base_per_chrom = seq_length*seq_per_chrom 
base_last_chrom = seq_length*seq_last_chrom
bscount = base_last_chrom + (base_per_chrom)*(chrom_num-1)


bs_tot_seq = seq_last_chrom + seq_per_chrom*(chrom_num-1)

print("Individual Sequence Length: " + str(seq_length))
print("Number of Experimental Sequences: " + str(sequence_num)) 
print("Number of Chromosomes: " + str(chrom_num))
print('')

print("Sequences per Chromosome: " + str(seq_per_chrom))
print("Sequences on Last Chromosome: " + str(seq_last_chrom))
print('')

print("Bases per Chromosome: " + str(base_per_chrom))
print("Bases on Last Chromosome: " + str(base_last_chrom))
print("Total Bases in Experiment: " + str(tot_bases))
if tot_bases-bscount != 0:
    print('Error! The total number of bases does not equal the sum of the bases divided into chromosomes!')
    sys.exit(1)
else:
    continue






Individual Sequence Length: 3021
Number of Experimental Sequences: 5073
Number of Chromosomes: 10

Sequences per Chromosome: 507
Sequences on Last Chromosome: 510

Bases per Chromosome: 1531647
Bases on Last Chromosome: 1540710
Total Bases in Experiment: 15325533


SyntaxError: 'continue' not properly in loop (<ipython-input-87-12598edc156e>, line 34)

In [ ]:
## Flattening classes for main
### Run this as the func in main that calls in md_functions
#for simulated only !
import os
import sys
from os import path
import pandas as pd
import csv

def mdcalc(window, threads, outdir, sample):
    windowbed_name = outdir + '/annotations/' + sample + '_simulated_window.bed' 
    input_file_statinfo = os.stat(windowbed_name)    
    if input_file_statinfo.st_size !=0:
        windowbed = open(windowbed_name)
        windowbed_csv_reader = csv.reader(windowbed, delimiter='\t')
        windowbed_line = next(windowbed_csv_reader)
        windowbed_peak_count = 0        
    else:
        raise ValueError("The windowed annotation file (sample_simulated_window.bed) is empty!")

    chromosomes = outdir + '/generated_sequences/' + sample + '_simulated.chrom.sizes'
    if chromosomes:
        chr_df = pd.read_csv(chromosomes, header=None, comment='#', sep="\t", usecols=[0,1], \
                                  names=['chrom', 'size'], na_filter=False, dtype={'chrom':'str', 'size':'int'})
        chr_list = list(chr_df['chrom'])
        CHROMOSOMES = [word for word in chr_list if len(word) <= 6]
    else:
        raise ValueError("Chromosome size file missing from generated_sequences(sample_simulated.chrom.sizes)!") 

    windowbed.seek(0)
    windowbed_csv_reader = csv.reader(windowbed, delimiter='\t')
    windowbed_line = next(windowbed_csv_reader)
    while(windowbed_line[0][0] == '#'):
        windowbed_line = next(windowbed_csv_reader)    

    motif_stats = []
    sorted_motif_stats = []
    tf_motif_path = outdir + '/motifs/simulated/*'
    motif_filenames = glob.glob(tf_motif_path)
    motif_count = len(motif_filenames)
    print("Processing %s motif files in %s" % motif_count, tf_motif_path)
    for filename in motif_filenames:
        filename_no_path = filename.split('/')[-1]
        if os.path.getsize(filename) > 0 and os.path.basename(filename).endswith(tuple(['.bed'])):
            [md_score, small_window, large_window, motif_site_count, heat] = get_md_score(filename, int(threads), windowbed_name, CHROMOSOMES, window)
            print('The MD-score for the provided BED regions vs %s is %.6f' % (filename_no_path, md_score))
            motif_stats.append({ 'motif_file': filename_no_path,'md_score': md_score, 'small_window': small_window, 'large_window': large_window, 'motif_site_count': motif_site_count, 'heat': heat })
    # sort the stats dictionary by MD-score, descending order
    sorted_motif_stats = sorted(motif_stats, key=itemgetter('md_score'), reverse=True)

    md_score_fp = open("%s/final_output/%s_simulated_md_scores.txt" % (outdir, sample), 'w')
    for stat in sorted_motif_stats:
        md_score_fp.write("%s,%s,%s,%s,%s,%s\n" % (stat['motif_file'], stat['md_score'], stat['small_window'], \
                           stat['large_window'], stat['motif_site_count'], stat['heat']))
    md_score_fp.close()

In [ ]:
def mdcalc_prescan(annotation, window, threads, outdir, sample, genome, pre_scan):
    windowbed_name = annotation 
    input_file_statinfo = os.stat(windowbed_name)    
    if input_file_statinfo.st_size !=0:
        windowbed = open(windowbed_name)
        windowbed_csv_reader = csv.reader(windowbed, delimiter='\t')
        windowbed_line = next(windowbed_csv_reader)
        windowbed_peak_count = 0        
    else:
        raise ValueError("The annotation is empty!")

        
    g = genome.split('/')[-1].split('.')[-2]    
    chromosomes = outdir + "/generated_sequences/" + g + ".chrom.sizes"
    if chromosomes:
        chr_df = pd.read_csv(chromosomes, header=None, comment='#', sep="\t", usecols=[0,1], names=['chrom', 'size'], na_filter=False, dtype={'chrom':'str', 'size':'int'})
        chr_list = list(chr_df['chrom'])
        CHROMOSOMES = [word for word in chr_list if len(word) <= 6]
    else:
        raise ValueError("Chromosome size file missing from generated_sequences(genome.chrom.sizes)!") 

    windowbed.seek(0)
    windowbed_csv_reader = csv.reader(windowbed, delimiter='\t')
    windowbed_line = next(windowbed_csv_reader)
    while(windowbed_line[0][0] == '#'):
        windowbed_line = next(windowbed_csv_reader)    

    motif_stats = []
    sorted_motif_stats = []
    tf_motif_path = pre_scan + '/*'
    motif_filenames = glob.glob(tf_motif_path)
    motif_count = len(motif_filenames)
    print("Processing %s motif files in %s" % motif_count, tf_motif_path)
    for filename in motif_filenames:
        filename_no_path = filename.split('/')[-1]
        if os.path.getsize(filename) > 0 and os.path.basename(filename).endswith(tuple(['.bed'])):
            [md_score, small_window, large_window, motif_site_count, heat] = get_md_score(filename, int(threads), windowbed_name, CHROMOSOMES, window)
            print('The MD-score for the provided BED regions vs %s is %.6f' % (filename_no_path, md_score))
            motif_stats.append({ 'motif_file': filename_no_path,'md_score': md_score, 'small_window': small_window, 'large_window': large_window, 'motif_site_count': motif_site_count, 'heat': heat })
    # sort the stats dictionary by MD-score, descending order
    sorted_motif_stats = sorted(motif_stats, key=itemgetter('md_score'), reverse=True)

    md_score_fp = open("%s/final_output/%s_experimental_md_scores.txt" % (outdir, sample), 'w')
    for stat in sorted_motif_stats:
        md_score_fp.write("%s,%s,%s,%s,%s,%s\n" % (stat['motif_file'], stat['md_score'], stat['small_window'], stat['large_window'], stat['motif_site_count'], stat['heat']))
    md_score_fp.close()


In [ ]:
## Additional MD Code for prescan
window, genome for args
if os.path.getsize(filename) > 0 and os.path.basename(filename).endswith(tuple(['.bed', '.BedGraph', '.txt']
##do this for the -p flag !
    if not os.listdir(motifs):
        raise NameError("The specified motif directory is empty or does not exist. Please check that there are motif files in the directory specified by the -p/--pre_scan argument.")

In [50]:
import math
##def expt_bed and sim_bed need to be fixed.
# '''This function takes the center of all sequences and generates a bedfile for dowindownstream use
# '''
outdir = '/scratch/Users/tajo5912/rbg/test'
sample = 'test'
window = 1500
chrom_num=10


df = pd.read_csv(outdir + "/generated_sequences/" + sample + "_experimental_window_sequences.fa", header=None)
df = df[df.index % 2 != 0]

l_expt_seq = df.values.tolist()
flat_list = [item for sublist in l_expt_seq for item in sublist]
strN = str('N')*20
file = ["{}{}".format(i,strN) for i in flat_list]

sequence_num = len(file)
seq_length = ((2*int(window))+21)
tot_bases = int(sequence_num*seq_length)
seq_per_chrom = math.floor(sequence_num/chrom_num)
remainder_seq_per_chrom = (sequence_num%chrom_num)
print(remainder_seq_per_chrom)
print(seq_per_chrom)
base_per_chrom = int(tot_bases/chrom_num) 
print(base_per_chrom)

# dbed = {}
# for i in range(0,(int(seq_per_chrom))):
#         dbed['r_' + str(i)] = [int(window-1)+i*(2*window+21), int(window+1)+i*(2*window+21)]
# dfbed = pd.DataFrame.from_dict(dbed, orient='index')
# dfbed.columns = ['start', 'stop']
# dfbed = dfbed.reset_index()
# dfbed = pd.concat([dfbed]*chrom_num)
# dfbed
# l_expt=[]
# for i in range(int(seq_per_chrom)):
#     for c in range(int(chrom_num)):
#         l_expt.append(str('exp_' + str(c+1)))
# l_expt = sorted(l_expt)
# len(l_expt)
# dfbed['exp'] = l_expt
# dfbed = dfbed[['exp', 'start', 'stop']]
# dfbed
# #print(chrom_num)
# #dfbed.to_csv(outdir + "/annotations/" + str(sample) + "_experimental_genome_centered.bed", header=None, index=False, sep='\t')

# # #creating the windowed bed

# dfbed["start_new"] = dfbed.apply(lambda x: round((x["start"] + x["stop"])/2), axis=1)
# dfbed["stop_new"] = dfbed.apply(lambda x: x["start_new"] + 1, axis = 1)

# ##the -1500 position from "origin"
# dfbed["start"] = dfbed.apply(lambda x: x["start_new"] - int(window), axis=1)

# ##the 1500 position from the "origin"
# dfbed["stop"] = dfbed.apply(lambda x: x["stop_new"] + int(window), axis=1)

# ##saving the new annotation
# dfbed.to_csv(outdir + '/annotations/' + sample + '_experimental_genome_window.bed', sep='\t', columns=["exp","start","stop"], header = False, index = False)
# print('Experimental Bed Files Genereated: ' + outdir + "/annotations/" + str(sample) + "_experimental_genome_centered.bed and " + str(sample) + "_experimental_genome_window.bed")

1
4412
13328954


In [ ]:
###SIMULATED
'''This function takes the center of all simulated sequences and generates a bedfile for dowindownstream use
'''
seq_per_chrom = int(self.sequence_num/self.chrom_num)
dbed = {}
for i in range(0,(int(seq_per_chrom))):
        dbed['r_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]
dfbed = pd.DataFrame.from_dict(dbed, orient='index')
dfbed.columns = ['start', 'stop']
dfbed = dfbed.reset_index()
dfbed = pd.concat([dfbed]*self.chrom_num)

l_sim=[]
for i in range(int(seq_per_chrom)):
    for c in range(int(self.chrom_num)):
        l_sim.append(str('sim_' + str(c+1)))
l_sim = sorted(l_sim)
dfbed['sim'] = l_sim
dfbed = dfbed[['sim', 'start', 'stop']]
dfbed.to_csv(self.outdir + "/annotations/" + str(self.sample) + "_simulated_centered.bed", header=None, index=False, sep='\t')

#creating the windowed bed

dfbed["start_new"] = dfbed.apply(lambda x: round((x["start"] + x["stop"])/2), axis=1)
dfbed["stop_new"] = dfbed.apply(lambda x: x["start_new"] + 1, axis = 1)

##the -1500 position from "origin"
dfbed["start"] = dfbed.apply(lambda x: x["start_new"] - int(self.window), axis=1)

##the 1500 position from the "origin"
dfbed["stop"] = dfbed.apply(lambda x: x["stop_new"] + int(self.window), axis=1)

##saving the new annotation
dfbed.to_csv(self.outdir + '/annotations/' + self.sample + '_simulated_window.bed', sep='\t', columns=["sim","start","stop"], header = False, index = False)
return print('Simulated Bed Files Genereated: ' + self.outdir + "/annotations/" + str(self.sample) + "_simulated_centered.bed and " + str(self.sample) + "_simulated_window.bed")

In [9]:

def identifier(row):
    return str(row['sequence_name']) + ':' + str(row['start']) + '-' + str(row['stop']) + '_' + str(row['strand'] + ';' + str(row['motif_id']))

def percent_gc(row):
    r = pd.Series(row['matched_sequence'])
    z = r.str.count('G') + r.str.count('C')
    x = z/len(row['matched_sequence'])
    return x

motif_dir = '/scratch/Users/tajo5912/rbg/levandowski2021/temp/sim_fimo_out/TP53_M06704_1'
motif_name = motif_dir.split('/')[-1]
df = pd.read_csv('%s/fimo.tsv' % (motif_dir), sep ='\t')
print("Post-processing FIMO output for %s" % motif_name)
df.drop(df.tail(3).index,inplace=True)

df = df.sort_values('sequence_name').reset_index()
df['identifier'] = df.apply(lambda row: identifier(row), axis=1)
df['percent_gc'] = df.apply(lambda row: percent_gc(row), axis=1)
df['mean_percent_gc'] = df['percent_gc'].mean()
df = df[['sequence_name', 'start', 'stop', 'identifier', 'motif_id','score', 'strand', 
         'matched_sequence', 'percent_gc', 'mean_percent_gc']]
#df=df.convert_dtypes()
df['start'] = df['start'].astype(int) 
df['stop'] = df['stop'].astype(int) 
df

Post-processing FIMO output for TP53_M06704_1


,sequence_name,start,stop,identifier,motif_id,score,strand,matched_sequence,percent_gc,mean_percent_gc
0,sim_1,1607701,1607720,sim_1:1607701.0-1607720.0_+;TP53_M06704_1,TP53_M06704_1,17.23,+,AGGCTTGCAAGGACATGCCT,0.55,0.533894
1,sim_1,12024196,12024215,sim_1:12024196.0-12024215.0_+;TP53_M06704_1,TP53_M06704_1,19.33,+,CGCCATGCCGGGACATGCCT,0.70,0.533894
2,sim_1,2943101,2943120,sim_1:2943101.0-2943120.0_-;TP53_M06704_1,TP53_M06704_1,17.14,-,AAACTTGCTCAGACATGACT,0.40,0.533894
3,sim_1,5926733,5926752,sim_1:5926733.0-5926752.0_-;TP53_M06704_1,TP53_M06704_1,17.90,-,CAAGATGCTCAGACATGTTT,0.40,0.533894
4,sim_1,4174561,4174580,sim_1:4174561.0-4174580.0_+;TP53_M06704_1,TP53_M06704_1,19.17,+,TTATATGCTCAGACATGTCT,0.35,0.533894
5,sim_1,8189020,8189039,sim_1:8189020.0-8189039.0_+;TP53_M06704_1,TP53_M06704_1,17.28,+,ATACAAGGTCAGGCATGCCC,0.55,0.533894
6,sim_1,11055590,11055609,sim_1:11055590.0-11055609.0_+;TP53_M06704_1,TP53_M06704_1,17.31,+,CGACATGTATAGACTTGTTC,0.40,0.533894
7,sim_1,5063863,5063882,sim_1:5063863.0-5063882.0_+;TP53_M06704_1,TP53_M06704_1,19.82,+,GTACTTGTCCAAGCATGTCC,0.50,0.533894
8,sim_1,7230857,7230876,sim_1:7230857.0-7230876.0_-;TP53_M06704_1,TP53_M06704_1,18.80,-,AGACTTGTTTGAGCATGTCC,0.45,0.533894
9,sim_1,148944,148963,sim_1:148944.0-148963.0_-;TP53_M06704_1,TP53_M06704_1,20.00,-,GGCCATGCTGGGACATGTCC,0.65,0.533894


In [41]:
# def fimotobed(outdir):
#     def identifier(row):
#         return str(row['sequence_name']) + ':' + str(row['start']) + '-' + str(row['stop']) + '_' + str(row['strand'] + ';' + str(row['motif_id']))

#     def percent_gc(row):
#         r = pd.Series(row['matched_sequence'])
#         z = r.str.count('G') + r.str.count('C')
#         x = z/len(row['matched_sequence'])
#         return x

#     motif_dirs = glob.glob(outdir + '/temp/sim_fimo_out/*')
#     for motif_dir in motif_dirs:
#         if path.isdir(motif_dir):
#             motif_name = motif_dir.split('/')[-1]
#             df = pd.read_csv('%s/fimo.tsv' % (motif_dir), sep ='\t')
#             print("Post-processing FIMO output for %s" % motif_name)
#             df.drop(df.tail(3).index,inplace=True)
#             if df.empty == True:
#                 print('Skipping ' + motif_name + ' -no motif hits.')
#             else:
#                 df = df.sort_values('sequence_name').reset_index()
#                 df['identifier'] = df.apply(lambda row: identifier(row), axis=1)
#                 df['percent_gc'] = df.apply(lambda row: percent_gc(row), axis=1)
#                 df['mean_percent_gc'] = df['percent_gc'].mean()
#                 df=df.convert_dtypes()
#                 df = df[['sequence_name', 'start', 'stop', 'identifier', 'score', 'strand', 
#                          'matched_sequence', 'percent_gc', 'mean_percent_gc']]
#                 df.to_csv(outdir + '/motifs/simulated/' + motif_name + '.sorted.bed', sep='\t', header=None, index=False)

#fimotobed('/scratch/Users/tajo5912/rbg/test')

In [ ]:
def expt_bed(self):
    #getting the sequence number
    annotation_file = pd.read_csv(self.annotation, header=None)
    sequence_num = len(annotation_file)





    #creating the locations for the all chromosomes except the last
    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(self.chrom_num-1))
    #adding the chromosome name to the genomic location
    l_expt=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(self.chrom_num-1)):
            l_expt.append(str('expt_' + str(c+1)))
    l_expt = sorted(l_expt)
    dfbed['expt'] = l_expt

    
    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(self.chrom_num-1))
    #adding the chromosome name to the genomic location
    l_sim=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(self.chrom_num-1)):
            l_sim.append(str('sim_' + str(c+1)))
    l_sim = sorted(l_sim)
    dfbed['sim'] = l_sim
    
    
    
    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('expt_' + str(self.chrom_num)))
    dflast['expt'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['expt', 'start', 'stop']]
    
    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('sim_' + str(self.chrom_num)))
    dflast['sim'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['sim', 'start', 'stop']]
    
    
    
    
    #saving the new annotation
    df.to_csv(self.outdir + "/annotations/" + str(self.sample) + "_experimental_genome_centered.bed", header=None, index=False, sep='\t')
    ##saving the new annotation
    df.to_csv(self.outdir + '/annotations/' + self.sample + '_experimental_genome_self.window.bed', sep='\t', columns=["expt","start","stop"], header = False, index = False)    
    return print('Experimental Bed Files Genereated: ' + self.outdir + "/annotations/" + str(self.sample) + "_experimental_genome_centered.bed and " + str(self.sample) + "_experimental_genome_self.window.bed")



def generate_bed(self):
    if self.experimental_fimo == True:
        #getting the sequence number
        annotation_file = pd.read_csv(self.annotation, header=None)
        sequence_num = len(annotation_file)
    else:
        sequence_num = self.sequence_num      

    #creating the locations for the all chromosomes except the last
    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(self.chrom_num-1))
    #adding the chromosome name to the genomic location
    l_sim=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(self.chrom_num-1)):
            l_sim.append(str('sim_' + str(c+1)))
    l_sim = sorted(l_sim)
    dfbed['sim'] = l_sim

    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('sim_' + str(self.chrom_num)))
    dflast['sim'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['sim', 'start', 'stop']]
    #saving the new annotation
    df.to_csv(self.outdir + "/annotations/" + str(self.sample) + "_simulated_centered.bed", header=None, index=False, sep='\t')



    ##saving the new annotation
    df.to_csv(self.outdir + '/annotations/' + self.sample + '_simulated_window.bed', sep='\t', columns=["sim","start","stop"], header = False, index = False)
    return print('Simulated Bed Files Genereated: ' + self.outdir + "/annotations/" + str(self.sample) + "_simulated_centered.bed and " + str(self.sample) + "_simulated_self.window.bed")